In [9]:
import pandas as pd
import numpy as np
import Utilidades as ut

In [4]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_csv("Archivos/Train_TP2_Datos_2020-2C.csv")
test = pd.read_csv("Archivos/Test_TP2_Datos_2020-2C.csv")

In [10]:
ut.diagnostico_df(entrenamiento)

Suma: 4999, Columna: Territory
Suma: 9497, Columna: Source 
Suma: 27, Columna: Billing_Country
Suma: 6973, Columna: Sales_Contract_No
Suma: 112, Columna: Account_Type
Suma: 15911, Columna: Brand
Suma: 15935, Columna: Product_Type
Suma: 15967, Columna: Size
Suma: 15928, Columna: Product_Category_B
Suma: 15982, Columna: Price
Suma: 16052, Columna: Currency
Suma: 16947, Columna: Last_Activity
Suma: 4625, Columna: Quote_Expiry_Date
Suma: 3209, Columna: ASP
Suma: 3209, Columna: ASP_(converted)
Suma: 75, Columna: Planned_Delivery_End_Date
Suma: 16947, Columna: Actual_Delivery_Date
Suma: 59, Columna: Total_Amount
Suma: 16947, Columna: Prod_Category_A


1. Completar los Territory incompletos que tienen como region a Japan, con Japan

In [3]:
#FILTRADO DE OPORTUNIDADES QUE NO SON CLOSE WON NI CLOSE LOST

entrenamiento = entrenamiento[( entrenamiento['Stage'] == 'Closed Won') | ( entrenamiento['Stage'] == 'Closed Lost')]

In [4]:
#FORMATO FECHAS

#Respalda fecha, usada para separa entrenamiento y test

columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']

def formato_fechas(x):
    for columna in columnas_fecha:
        x[columna] = pd.to_datetime(x[columna])
        
formato_fechas(entrenamiento)
formato_fechas(test)

In [5]:
#ORDENO POR FECHA AMBOS DATAFRAMES

entrenamiento = entrenamiento.sort_values(by=['Opportunity_Created_Date'])
test = test.sort_values(by=['Opportunity_Created_Date'])

In [6]:
#RECUPERACION Y CONVERSION DE COLUMNAS

def recuperacion_y_conversion(df):
    #Arreglo el nombre de la columna 'Source'
    df = df.rename(columns={'Source ' : 'Source'})

    # Convierto Total_Amount y Total_Taxable_Amount a dolares, filtro columnas innecesarias (codigo extraido de TP1)

    df['Convertion_Factor'] = df['ASP_(converted)'] / df['ASP']

    #Intentamos salvar algunas entradas reemplazando con la media de la oportunidad

    df['Convertion_Factor_AVG'] = df.groupby('Opportunity_ID')['Convertion_Factor'].transform('mean')
    df['Convertion_Factor_is_null'] = df['Convertion_Factor'].isnull()
    df['Convertion_Factor'] = np.where(df['Convertion_Factor_is_null'] == 1, df['Convertion_Factor_AVG'], df['Convertion_Factor'])
    df['Convertion_Factor_AVG'] = df.groupby('ASP_Currency')['Convertion_Factor'].transform('mean')
    df['Convertion_Factor_is_null'] = df['Convertion_Factor'].isnull()
    df['Convertion_Factor'] = np.where(df['Convertion_Factor_is_null'] == 1, df['Convertion_Factor_AVG'], df['Convertion_Factor'])

    #Eliminamos las entradas que no pudimos salvar.
    df = df.loc[df.Convertion_Factor.isnull() == 0]

    #Eliminamos infinitos
    df = df.loc[df.Convertion_Factor != np.inf]

    # Eliminamos todos los 'Opportunity_ID' que tienen algun registro con Total_Amount == None 
    df['Total_Amount_is_null'] = df.Total_Amount.isnull().astype(int)
    df['Total_Amount_is_null'] = df.groupby('Opportunity_ID')['Total_Amount_is_null'].transform('sum')
    df = df.loc[df.Total_Amount_is_null == 0]

    #Convertimos los valores a dolares
    df['Total_Amount(USD)'] = df['Total_Amount'] * df['Convertion_Factor']
    df['Total_Taxable_Amount(USD)'] = df['Total_Taxable_Amount'] * df['Convertion_Factor']

    #Rescatamos ASP
    df['ASP_(converted)_avg'] = df.groupby('Region')['ASP_(converted)'].transform('mean')
    df['ASP_(converted)_is_null'] = df['ASP_(converted)'].isnull()
    df['ASP_(converted)'] = np.where(df['ASP_(converted)_is_null'] == 1, df['ASP_(converted)_avg'], df['ASP_(converted)'])

    #Eliminamos columnas utilizadas

    df = df.drop(columns={'Total_Amount_is_null', 
                          'Convertion_Factor_AVG',
                          'Convertion_Factor',
                          'Convertion_Factor_is_null',
                          'ASP_(converted)_avg',
                          'ASP_(converted)_is_null'})
    
    # Intentamos recuperar los datos faltantes

    #ACA LE PUEDO HABER PIFIADO
        
    # Billing Country y #Account_Type
    idx = df[df.Billing_Country == 'None'].index

    # Se completan los campos faltantes en base a la moda de la region.
    # Se intentaron completar a partir del nombre de cuenta, nombre de oportunidad e id de oportunidad, pero solo aparecia esta entrada.
    df.loc[idx, 'Billing_Country'] = df[df.Region == 'APAC']['Billing_Country'].mode()[0]
    df.loc[idx, 'Account_Type'] = df[df.Region == 'APAC']['Account_Type'].mode()[0]

    # Planned_Delivery_End_Date

    # Ya que son oportunidades de un unico producto, podemos predecir una fecha de fin de entrega a partir de la media de la region y familia de producto

    df['Planned_Delivery_Duration'] = (df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']) / np.timedelta64(1, 'D')
    df['Planned_Delivery_Duration_AVG'] = np.round(df.groupby(['Region', 'Product_Family'])['Planned_Delivery_Duration'].transform('mean'))
    df['Planned_Delivery_Duration_AVG'] = pd.to_timedelta(df['Planned_Delivery_Duration_AVG'], 'D')
    df['Predicted_Planned_Delivery_End_Date'] = df['Planned_Delivery_Start_Date'] + df['Planned_Delivery_Duration_AVG']

    df['Planned_Delivery_End_Date_is_null'] = df['Planned_Delivery_End_Date'].isnull()
    df['Planned_Delivery_End_Date'] = np.where(df['Planned_Delivery_End_Date_is_null'] == 1, df['Predicted_Planned_Delivery_End_Date'], df['Planned_Delivery_End_Date'])

    # Completamos las que quedaron con la media de la region, sin tener en cuenta el tipo de producto

    df['Planned_Delivery_Duration_AVG'] = round(df.groupby('Region')['Planned_Delivery_Duration'].transform('mean'))
    df['Planned_Delivery_Duration_AVG'] = pd.to_timedelta(df['Planned_Delivery_Duration_AVG'], 'D')
    df['Predicted_Planned_Delivery_End_Date'] = df['Planned_Delivery_Start_Date'] + df['Planned_Delivery_Duration_AVG']

    df['Planned_Delivery_End_Date_is_null'] = df['Planned_Delivery_End_Date'].isnull()
    df['Planned_Delivery_End_Date'] = np.where(df['Planned_Delivery_End_Date_is_null'] == 1, df['Predicted_Planned_Delivery_End_Date'], df['Planned_Delivery_End_Date'])

    df = df.drop(columns = ['Planned_Delivery_Duration',
                                      'Planned_Delivery_Duration_AVG',
                                      'Predicted_Planned_Delivery_End_Date', 
                                      'Planned_Delivery_End_Date_is_null'])
    
    return df

entrenamiento = recuperacion_y_conversion(entrenamiento)
test = recuperacion_y_conversion(test)

In [7]:
#LIMPIEZA COLUMNAS IRRECUPERABLES

entrenamiento = entrenamiento.drop(columns=['ASP_(converted)_Currency','Quote_Type','Brand','Product_Type','Size','Product_Category_B','Price','Currency','Last_Activity','Actual_Delivery_Date','Prod_Category_A'])
test = test.drop(columns=['ASP_(converted)_Currency','Quote_Type','Brand','Product_Type','Size','Product_Category_B','Price','Currency','Last_Activity','Actual_Delivery_Date','Prod_Category_A'])

In [8]:
# COLUMNAS OBJECT A CATEGORY

def object_a_category(df):
    columnas_object = list(df.select_dtypes(include=['object']).columns)
    for columna in columnas_object:
        df[columna] = df[columna].astype('category')
    return df

entrenamiento = object_a_category(entrenamiento)
test = object_a_category(test)

In [9]:
# GUARDADO ARCHIVOS TEMPORALES
entrenamiento.to_pickle("Archivos/Entrenamiento_limpio.pkl")
test.to_pickle("Archivos/Test_limpio.pkl")

In [10]:
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16596 entries, 16104 to 16102
Data columns (total 43 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  16596 non-null  int64         
 1   Region                              16596 non-null  category      
 2   Territory                           16596 non-null  category      
 3   Pricing, Delivery_Terms_Quote_Appr  16596 non-null  int64         
 4   Pricing, Delivery_Terms_Approved    16596 non-null  int64         
 5   Bureaucratic_Code_0_Approval        16596 non-null  int64         
 6   Bureaucratic_Code_0_Approved        16596 non-null  int64         
 7   Submitted_for_Approval              16596 non-null  int64         
 8   Bureaucratic_Code                   16596 non-null  category      
 9   Account_Created_Date                16596 non-null  datetime64[ns]
 10  Source            

In [12]:
entrenamiento.dtypes

ID                                             int64
Region                                      category
Territory                                   category
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Submitted_for_Approval                         int64
Bureaucratic_Code                           category
Account_Created_Date                  datetime64[ns]
Source                                      category
Billing_Country                             category
Account_Name                                category
Opportunity_Name                            category
Opportunity_ID                                 int64
Sales_Contract_No                           category
Account_Owner                               category
Opportunity_Owner                           category
Account_Type                                ca

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2551 entries, 718 to 2550
Data columns (total 42 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  2551 non-null   int64         
 1   Region                              2551 non-null   category      
 2   Territory                           2551 non-null   category      
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64         
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64         
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64         
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64         
 7   Submitted_for_Approval              2551 non-null   int64         
 8   Bureaucratic_Code                   2551 non-null   category      
 9   Account_Created_Date                2551 non-null   datetime64[ns]
 10  Source                

In [16]:
test.dtypes

ID                                             int64
Region                                      category
Territory                                   category
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Submitted_for_Approval                         int64
Bureaucratic_Code                           category
Account_Created_Date                  datetime64[ns]
Source                                      category
Billing_Country                             category
Account_Name                                category
Opportunity_Name                            category
Opportunity_ID                                 int64
Sales_Contract_No                           category
Account_Owner                               category
Opportunity_Owner                           category
Account_Type                                ca

In [20]:
entrenamiento['Month'].dt.month

16104    10
16105     5
16106     8
16107     7
16109     1
         ..
16099     2
16100    12
16103     1
16101     1
16102     1
Name: Month, Length: 16596, dtype: int64